In [2]:
!pip install fairseq sentencepiece sacremoses transformers tensorboardX

In [ ]:
import os
os.chdir("/content/drive/MyDrive/data")

In [7]:
import sentencepiece as spm
from glob import glob

In [8]:
spm.SentencePieceTrainer.train(
    "--input=./data/tokenized_data/train.en --model_prefix=bpe_en --vocab_size=5000 --character_coverage=1.0 --model_type=bpe"
)

spm.SentencePieceTrainer.train(
    "--input=./data/tokenized_data/train.fa --model_prefix=bpe_fa --vocab_size=5000 --character_coverage=1.0 --model_type=bpe"
)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=./data/tokenized_data/train.en --model_prefix=bpe_en --vocab_size=5000 --character_coverage=1.0 --model_type=bpe
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./data/tokenized_data/train.en
  input_format: 
  model_prefix: bpe_en
  model_type: BPE
  vocab_size: 5000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
 

In [ ]:
!mkdir -p ./data/bpe_data

In [24]:
sp_bpe_en = spm.SentencePieceProcessor()
sp_bpe_en.load('bpe_en.model')
for data_dir in glob('./data/tokenized_data/*.en'):
    i = 0
    print(data_dir)
    processed_data = ""
    with open(data_dir, 'r') as f:
        for line in f:
            p = sp_bpe_en.encode_as_pieces(line)
            processed_data += " ".join(p) + "\n"
    with open(os.path.join('./data/bpe_data', os.path.basename(data_dir)), 'w') as f:
        f.write(processed_data)

./data/tokenized_data/valid.en
./data/tokenized_data/train.en
./data/tokenized_data/test.en


In [25]:

sp_bpe_fa = spm.SentencePieceProcessor()
sp_bpe_fa.load('bpe_fa.model')
for data_dir in glob('./data/tokenized_data/*.fa'):
    i = 0
    print(data_dir)
    processed_data = ""
    with open(data_dir, 'r') as f:
        for line in f:
            p = sp_bpe_en.encode_as_pieces(line)
            processed_data += " ".join(p) + "\n"
    with open(os.path.join('./data/bpe_data', os.path.basename(data_dir)), 'w') as f:
        f.write(processed_data)

./data/tokenized_data/valid.fa
./data/tokenized_data/test.fa
./data/tokenized_data/train.fa


In [ ]:
!mkdir -p ./data/data_bin_bpe

!fairseq-preprocess --source-lang en --target-lang fa \
  --trainpref ./data/bpe_data/train \
  --validpref ./data/bpe_data/valid \
  --testpref ./data/bpe_data/test \
  --nwordstgt 40000 \
  --nwordssrc 40000 \
  --destdir ./data/data_bin_bpe/

In [ ]:
!fairseq-train \
    "./data/data_bin_bpe/" \
    --arch lstm --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 0.0025 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.25 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --fp16 --memory-efficient-fp16 \
    --max-epoch 5 \
    --save-dir ./data/data_bin_bpe/checkpoints/ \
    --tensorboard-logdir ./data/data_bin_bpe/logs

In [ ]:
!fairseq-generate \
    "./data/data_bin_bpe" \
    --batch-size 128 \
    --path "./data/data_bin_bpe/checkpoints/checkpoint_best.pt" \
    --beam 5 > "./data/data_bin_bpe/new_eval.txt"